In [2]:
import numpy as np
from os.path import join
from itertools import product

from brainpipe.system import study
from brainpipe.feature import PLV

from utils import subjects

In [ ]:
"""
Compute PLV btw HC and OFC in patients with both ROIs
>>> Try all frequency bands [theta to gamma]
"""

st = study('Olfacto')
path_data = join(st.path, 'database/Encoding_By_Cond_v=1_elecs=all/')
filename = join(path_data, '{}_odor_{}_E.npz')
save_path = join(st.path, 'feature/PLV_all_freqs_Encoding_v=1_elecs=all/')
savename = join(save_path, '{}_plv_conds=odor_elecs={}_{}.npz')
conds = ['low','mid','high'] #to be concatenated
roi1, roi2 = 'OFC_olf', 'pPirT'

# Parameters for the PLV
f = [[3,7],[7,13],[13,30],[30,60],[60,120]] 
fname = ['theta','alpha','beta','gamma_low','gamma_high'] 
n_perm = 100
sf = 512
method = 'hilbert'

for su in subjects:
    
    #concatenate all conds on trial dimension
    all_data = np.array([])
    for cond in conds:
        mat = np.load(filename.format(su,cond),allow_pickle=True)
        all_data = np.concatenate((all_data,mat['x']),axis=-1) if np.size(all_data) else mat['x']

    #select only subjects with HC and OFC electrodes
    new_labs = ['pPirT' if x == 'Amg' else x for x in mat['Mai_RL'] ]
    log_su = [1 if all(i in new_labs for i in [roi1,roi2]) else 0]
    if log_su == [1]:
        idx_HC = [i for i,lab in enumerate(new_labs) if lab == roi1]
        idx_OFC = [i for i,lab in enumerate(new_labs) if lab == roi2]
        print('>> processing', su, len(idx_HC),'elecs in ',roi1,' &',
                len(idx_OFC), 'elecs in ',roi2)
                
        #compute PLV metrics between aHC and OFC electrodes
        data_e1 = all_data[idx_HC,:,:]
        data_e2 = all_data[idx_OFC,:,:]
        print(data_e1.shape,data_e2.shape)
        def_plv = PLV(sf=sf, npts=data_e1.shape[1], f=f, method=method, cycle=3)
        
        dico_plv = {}
        plv_vals,_ = def_plv.get(data_e1,data_e2,n_perm = n_perm)
        dico_plv['plv'] = plv_vals
        print(plv_vals.shape)
        
        for c in ['xyz','channels','Mai_RL']:
            dico_plv[c] = mat[c]
        np.savez(savename.format(su,roi1,roi2),**dico_plv)

In [ ]:
"""
Plot PLV values (nfreqs, nelecs1, nelecs2, npts)
"""
import matplotlib.pyplot as plt
from brainpipe.visual import addLines

st = study('Olfacto')
data_path = join(st.path, 'feature/PLV_all_freqs_Encoding_v=1_elecs=all/')
filename = join(data_path, '{}_plv_conds=all_elecs=mean_OFC_aHC.npz')
savename = join(data_path, '{}_{}_OFC={}_aHC=all_conds=all_meth=mean.png')

subjects = ['LEFC','PIRJ','SEMC','VACJ']

for su in subjects:
    mat = np.load(filename.format(su))
    plv = mat['plv']
    nfreqs, nelecs1, nelecs2, npts = plv.shape
    
    for freq in range(1):
        for elec in range(nelecs2): #OFC electrodes
            plv_plot = plv[freq,:,elec,:]
            time = np.arange(-1,2,1/512)
            
            #mask data not significant 
            pval = mat['p'][freq,:,elec,:]
            mask = pval.copy()
            sig_idx = mask < 0.05
            mask[sig_idx] = 1
            mask[np.invert(sig_idx)] = 0.6 #control for mask opcapity
            
            #plot PLV values
            title = su+' PLV between OFC and aHC in elec('+str(elec)+')'
            plvObj = PLV(sf=sf, npts=npts, f=f, method=method, cycle=3)
            plvObj.plot2D(plt.figure(figsize=(6,5)), plv_plot, cmap='viridis',
                          xvec=time, yvec=np.arange(1,nelecs1+2), xlabel='Time (s)',
                          ylabel='Electrodes', title=title,mask=mask,pltype='imshow',
                          cblabel='Phase Locking Value (PLV)',resample=(0,0))         
            addLines(plt.gca(), vLines=[0], vColor=['black'], vWidth=[2])
            
            plt.savefig(savename.format(freq,su,str(elec)))
            plt.clf()
            plt.close()

In [ ]:
"""
Plot PLV by FREQ with stats (nfreqs, nelecs1, nelecs2, npts)
"""
import matplotlib.pyplot as plt
from brainpipe.visual import addLines

st = study('Olfacto')
data_path = join(st.path, 'feature/PLV_all_freqs_Encoding_v=1_elecs=all/')
filename = join(data_path, '{}_plv_conds=all_elecs=OFC_aHC.npz')
savename = join(data_path, 'Freqs=all_{}_OFC=all_aHC=all_conds=all.png')

subjects = ['LEFC','PIRJ','SEMC','VACJ']

for su in subjects:
    mat = np.load(filename.format(su))
    plv = mat['plv']
    nfreqs, _, _, _ = plv.shape
    
    mean_plv, max_plv, min_pval = [], [], []
    for freq in range(nfreqs):
        mean_plv_freq = np.mean(plv[freq,:,:,:])
        max_plv_freq = np.max(np.mean(plv[freq,:,:,:],axis=(0,1)))
        p_min = np.min(np.mean(mat['p'][freq,:,:,:],axis=(0,1)))
        
        mean_plv.append(mean_plv_freq), max_plv.append(max_plv_freq)
        min_pval.append(p_min)
    
    #Plot average data by subjects
    title = su+' PLV between OFC and aHC ALL elecs ALL freqs'
    fig, (ax1, ax2, ax3) = plt.subplots(3,1)
    freq_vec = np.arange(nfreqs)
    fig.suptitle(title)
    ax1.plot(freq_vec, mean_plv)
    ax2.plot(freq_vec, max_plv)
    ax3.plot(freq_vec,np.log(min_pval),'-b')
    #ax3.plot(freq_vec,[np.log(0.01)]*len(freq_vec))
    plt.savefig(savename.format(su))

#### Compute PLV and Plot by condition

In [ ]:
"""
Compute PLV in patients with both ROIs
>>> Try all frequency bands [theta to gamma]
"""

st = study('Olfacto')
path_data = join(st.path, 'database/Encoding_No_Odor/')
#path_data = join(st.path, 'database/Encoding_By_Cond_v=1_elecs=all/2conds/')
#filename = join(path_data, '{}_odor_{}_E.npz')
filename = join(path_data, '{}_odor_no_odor_bipo_sel_physFT.npz')
save_path = join(st.path, 'feature/PLV_all_freqs_Encoding_v=1_elecs=all/')
savename = join(save_path, '{}_plv_conds={}_elecs={}_{}.npz')
conds = ['no_odor'] #to be concatenated

# Parameters for the PLV
f = [[3,7],[7,13],[13,30],[30,60],[60,120]] 
fname = ['theta','alpha','beta','gamma_low','gamma_high'] 
n_perm = 100
sf = 512
method = 'hilbert'
roi1, roi2 = 'aHC', 'pPirT' #'Amg'

for su in subjects:
    #concatenate all conds on trial dimension
    for cond in conds:
        mat = np.load(filename.format(su,cond),allow_pickle=True)

        #select only subjects with HC and OFC electrodes
        new_labs = ['pPirT' if x == 'Amg' else x for x in mat['Mai_RL'] ]
        log_su = [1 if all(i in new_labs for i in [roi1, roi2]) else 0]
        if log_su == [1]:
            idx_HC = [i for i,lab in enumerate(new_labs) if lab == roi1]
            idx_OFC = [i for i,lab in enumerate(new_labs) if lab == roi2]
            print('>> processing', su, len(idx_HC),'elecs in '+roi1+' &',
                    len(idx_OFC), 'elecs in '+roi2)

            #compute PLV metrics between aHC and OFC electrodes
            data_e1 = mat['x'][idx_HC,:,:]
            data_e2 = mat['x'][idx_OFC,:,:]
            def_plv = PLV(sf=sf, npts=data_e1.shape[1], f=f, method=method, cycle=3)

            dico_plv = {}
            dico_plv['plv'], dico_plv['p'] = def_plv.get(data_e1,data_e2,
                                                         n_perm = n_perm, n_jobs=-1)

            for c in ['xyz','channels','Mai_RL']:
                dico_plv[c] = mat[c]
            np.savez(savename.format(su,cond,roi1,roi2),**dico_plv)

In [ ]:
"""
Plot PLV values (nfreqs, nelecs1, nelecs2, npts)
"""
import matplotlib.pyplot as plt
from brainpipe.visual import addLines

st = study('Olfacto')
data_path = join(st.path, 'feature/PLV_all_freqs_Encoding_v=1_elecs=all/')
filename = join(data_path, '{}_plv_conds={}_elecs=mean_OFC_aHC.npz')
savename = join(data_path, '{}_{}_OFC=all_aHC=all_conds={}_meth=mean.png')

subjects = ['LEFC','PIRJ','SEMC','VACJ']

for su,cond in product(subjects,conds):
    
    mat = np.load(filename.format(su,cond))
    plv = mat['plv']
    nfreqs, nelecs1, nelecs2, npts = plv.shape
    
    for freq in range(1):
        for elec in range(nelecs2): #OFC electrodes
            plv_plot = plv[freq,:,elec,:]
            time = np.arange(-1,2,1/512)
            
            #mask data not significant 
            pval = mat['p'][freq,:,elec,:]
            mask = pval.copy()
            sig_idx = mask < 0.05
            mask[sig_idx] = 1
            mask[np.invert(sig_idx)] = 0.6 #control for mask opcapity
            
            #plot PLV values
            title = su+' PLV for '+cond+' between OFC and aHC in all elecs'
            plvObj = PLV(sf=sf, npts=npts, f=f, method=method, cycle=3)
            plvObj.plot2D(plt.figure(figsize=(6,5)), plv_plot, cmap='viridis',
                          xvec=time, yvec=np.arange(1,nelecs1+2), xlabel='Time (s)',
                          ylabel='Electrodes', title=title,mask=mask,pltype='imshow',
                          cblabel='Phase Locking Value (PLV)',resample=(0,0))         
            addLines(plt.gca(), vLines=[0], vColor=['black'], vWidth=[2])
            
            plt.savefig(savename.format(freq,su,cond))
            plt.clf()
            plt.close()

In [ ]:
"""
Compute statistical differences btw High and Low in 4 patients
average PLV across time and electrodes
"""
from scipy.stats import wilcoxon
from Ttests_perm_arthur import ttest_perm
from os.path import join
import numpy as np
from brainpipe.system import study

st = study('Olfacto')
data_rep = join(st.path,'feature/PLV_all_freqs_Encoding_v=1_elecs=all/2conds/')

to_take = [1536,2560] #[0;+2s]
freqs = ['theta','alpha','beta','gamma_low','gamma_high']
conds = ['no_odor','low']
files1 = st.search('conds='+conds[0]+'_elecs=aHC_OFC_olf', folder=data_rep)
files2 = st.search('conds='+conds[1]+'_elecs=aHC_OFC_olf', folder=data_rep)
mean = 'elecs'#'all'

for f,freq in enumerate(freqs):
    plv_mean0, plv_mean1 = np.array([]), np.array([])
    for f1,f2 in zip(files1,files2):
        print("analysis on {} subjects".format(len(files1)))
        plv0 = np.load(data_rep+f1)['plv'][f,:,:,to_take[0]:to_take[1]]
        plv1 = np.load(data_rep+f2)['plv'][f,:,:,to_take[0]:to_take[1]]
        if mean == 'all':
            plv0, plv1 = np.mean(plv0), np.mean(plv1)
        if mean == 'elecs':
            plv0 = np.reshape(np.max(plv0,axis=-1),(plv0.shape[0]*plv0.shape[1]))
            plv1 = np.reshape(np.max(plv1,axis=-1),(plv1.shape[0]*plv1.shape[1]))
        plv_mean0 = np.hstack((plv_mean0,plv0)) if np.size(plv_mean0) else plv0
        plv_mean1 = np.hstack((plv_mean1,plv1)) if np.size(plv_mean1) else plv1
    T2, p2 = wilcoxon(plv_mean0,plv_mean1, alternative='less')
    print(freq, 'all su', 'mean=',mean, T2, np.round(p2,4))
    print('mean values', np.mean(plv_mean0), np.mean(plv_mean1))
    print('')

In [ ]:
"""
Compute statistical differences btw 3 CONDITIONS
average PLV across time and electrodes
"""
from scipy.stats import wilcoxon
from Ttests_perm_arthur import ttest_perm
from os.path import join
import numpy as np
from brainpipe.system import study

st = study('Olfacto')
data_rep = join(st.path,'feature/PLV_all_freqs_Encoding_v=1_elecs=all/2conds/')

to_take = [1536,2560] #[0;+2s]
freqs = ['theta']#,'alpha','beta','gamma_low','gamma_high']
conds = ['no_odor','low','high']
roi1, roi2 = ['aHC','OFC_olf']
files1 = st.search('conds='+conds[0]+'_elecs='+roi1+'_'+roi2, folder=data_rep)
files2 = st.search('conds='+conds[1]+'_elecs='+roi1+'_'+roi2, folder=data_rep)
files3 = st.search('conds='+conds[2]+'_elecs='+roi1+'_'+roi2, folder=data_rep)
mean = 'elecs'#'all'

for f,freq in enumerate(freqs):
    plv_mean0, plv_mean1, plv_mean2 = np.array([]), np.array([]), np.array([])
    for f1,f2,f3 in zip(files1,files2,files3):
        print("analysis on {} subjects".format(len(files1)))
        plv0 = np.load(data_rep+f1)['plv'][f,:,:,to_take[0]:to_take[1]]
        plv1 = np.load(data_rep+f2)['plv'][f,:,:,to_take[0]:to_take[1]]
        plv2 = np.load(data_rep+f3)['plv'][f,:,:,to_take[0]:to_take[1]]
        if mean == 'all':
            plv0, plv1 = np.mean(plv0), np.mean(plv1)
        if mean == 'elecs':
            plv0 = np.reshape(np.mean(plv0,axis=-1),(plv0.shape[0]*plv0.shape[1]))
            plv1 = np.reshape(np.mean(plv1,axis=-1),(plv1.shape[0]*plv1.shape[1]))
            plv2 = np.reshape(np.mean(plv2,axis=-1),(plv2.shape[0]*plv2.shape[1]))
        plv_mean0 = np.hstack((plv_mean0,plv0)) if np.size(plv_mean0) else plv0
        plv_mean1 = np.hstack((plv_mean1,plv1)) if np.size(plv_mean1) else plv1
        plv_mean2 = np.hstack((plv_mean2,plv2)) if np.size(plv_mean2) else plv2
    T0, p0 = wilcoxon(plv_mean0,plv_mean1)#, alternative='less')
    T1, p1 = wilcoxon(plv_mean0,plv_mean2)#, alternative='less')
    T2, p2 = wilcoxon(plv_mean1,plv_mean2)#, alternative='less')
    print('/n',freq, 'all su')
    print('LOW MID', T0, np.round(p0,4))
    print('LOW HIGH', T1, np.round(p1,4))
    print('MID HIGH', T2, np.round(p2,4))
    print('/n mean values', np.mean(plv_mean0), np.mean(plv_mean1),np.mean(plv_mean2))


### Compute PLV Early / Late conditions

In [ ]:
"""
Compute PLV in patients with both ROIs
>>> Try all frequency bands [theta to gamma]
"""

st = study('Olfacto')
path_data = join(st.path, 'database/Encoding_By_Cond_v=1_elecs=all/')
filename = join(path_data, '{}_odor_cond={}_early_late.npz')
save_path = join(st.path, 'feature/PLV_all_freqs_Encoding_v=1_elecs=all/')
savename = join(save_path, '{}_plv_conds={}_early_late_elecs={}_{}.npz')

# Parameters for the PLV
f = [[3,7],[7,13],[13,30],[30,60],[60,120]] 
fname = ['theta','alpha','beta','gamma_low','gamma_high'] 
n_perm = 100
sf = 512
method = 'hilbert'
roi1, roi2 = 'aHC', 'pPirT'
cond = 'high'

for su in subjects:
    mat = np.load(filename.format(su,cond),allow_pickle=True)

    #select only subjects with HC and OFC electrodes
    new_labels = ['pPirT' if x == 'Amg' else x for x in mat['Mai_RL']]
    log_su = [1 if all(i in new_labels for i in [roi1, roi2]) else 0]
    if log_su == [1]:
        idx_HC = [i for i,lab in enumerate(new_labels) if lab == roi1]
        idx_OFC = [i for i,lab in enumerate(new_labels) if lab == roi2]
        print('>> processing', su, len(idx_HC),'elecs in '+roi1+' &',
                len(idx_OFC), 'elecs in '+roi2)

        #compute PLV metrics between aHC and OFC electrodes
        data_e1_E, data_e1_L = mat['x_E'][idx_HC,:,:], mat['x_L'][idx_HC,:,:]
        data_e2_E, data_e2_L = mat['x_E'][idx_OFC,:,:], mat['x_L'][idx_OFC,:,:]
        def_plv = PLV(sf=sf, npts=data_e1.shape[1], f=f, method=method, cycle=3)

        dico_plv = {}
        dico_plv['plv_E'], dico_plv['p_E'] = def_plv.get(data_e1_E,data_e2_E,
                                                     n_perm = n_perm)
        dico_plv['plv_L'], dico_plv['p_L'] = def_plv.get(data_e1_L,data_e2_L,
                                                     n_perm = n_perm)

        for c in ['xyz','channels','Mai_RL']:
            dico_plv[c] = mat[c]
        np.savez(savename.format(su,cond,roi1,roi2),**dico_plv)

In [ ]:
"""
Compute statistical differences btw EARLY & LATE
average PLV across time and electrodes
"""
from scipy.stats import wilcoxon
from Ttests_perm_arthur import ttest_perm
from os.path import join
import numpy as np
from brainpipe.system import study

st = study('Olfacto')
data_rep = join(st.path,'feature/PLV_all_freqs_Encoding_v=1_elecs=all/')

to_take = [1536,2560] #[0;+2s]
freqs = ['theta']#,'alpha','beta','gamma_low','gamma_high']
roi1, roi2 = ['aHC','OFC_olf']
files1 = st.search('high_early_late_elecs='+roi1+'_'+roi2, folder=data_rep)
mean = 'elecs'#'all'

for f,freq in enumerate(freqs):
    plv_mean0, plv_mean1 = np.array([]), np.array([])
    for f1 in files1:
        print("analysis on {} subjects".format(len(files1)))
        plv0 = np.load(data_rep+f1)['plv_E'][f,:,:,to_take[0]:to_take[1]]
        plv1 = np.load(data_rep+f1)['plv_L'][f,:,:,to_take[0]:to_take[1]]
        if mean == 'all':
            plv0, plv1 = np.mean(plv0), np.mean(plv1)
        if mean == 'elecs':
            plv0 = np.reshape(np.max(plv0,axis=-1),(plv0.shape[0]*plv0.shape[1]))
            plv1 = np.reshape(np.max(plv1,axis=-1),(plv1.shape[0]*plv1.shape[1]))
        plv_mean0 = np.hstack((plv_mean0,plv0)) if np.size(plv_mean0) else plv0
        plv_mean1 = np.hstack((plv_mean1,plv1)) if np.size(plv_mean1) else plv1
    T0, p0 = wilcoxon(plv_mean0,plv_mean1)#, alternative='less')
    print('/n',freq, 'all su')
    print('Early vs Late', T0, np.round(p0,4))
    print('/n mean values', np.mean(plv_mean0), np.mean(plv_mean1))


In [43]:
"""
Compute statistical differences btw EARLY & LATE
average PLV across time and electrodes
"""
from scipy.stats import wilcoxon
from Ttests_perm_arthur import ttest_perm
from os.path import join
import numpy as np
from brainpipe.system import study

st = study('Olfacto')
data_rep = join(st.path,'feature/PLV_all_freqs_Encoding_v=1_elecs=all/')
filename = join(data_rep, 'Early_Late/{}_plv_conds=early_late_elecs={}_{}.npz')

to_take = [1536,2560] #[0;+2s]
freqs = ['theta']#,'alpha','beta','gamma_low','gamma_high']
roi1, roi2 = ['aHC','OFC_olf']
subjects = ['LEFC','SEMC']

for f,freq in enumerate(freqs):
    plv_mean0, plv_mean1 = np.array([]), np.array([])
    for su in subjects:
        mat = np.load(filename.format(su, roi1,roi2))
        plv0 = mat['plv_E'][f,:,:,to_take[0]:to_take[1]]
        plv1 = mat['plv_L'][f,:,:,to_take[0]:to_take[1]]

        plv0 = np.reshape(np.mean(plv0,axis=-1),(plv0.shape[0]*plv0.shape[1]))
        plv1 = np.reshape(np.max(plv1,axis=-1),(plv1.shape[0]*plv1.shape[1]))

        T0, p0 = wilcoxon(plv0,plv1)#, alternative='less')
        print('/n',freq, su)
        print('Early vs Late', T0, np.round(p0,4))
        print(np.mean(plv0), np.mean(plv1))        
        print(np.std(plv0), np.std(plv1))

-> Olfacto loaded
/n theta LEFC
Early vs Late 0.0 0.0
0.17949113166620362 0.3642856498683701
0.02796307365233956 0.05752302780072473
/n theta SEMC
Early vs Late 0.0 0.0007
0.20872600389271143 0.4017713890754511
0.03482455968923194 0.0493291185034378


In [44]:
"""
Compute statistical differences btw 3 CONDITIONS
average PLV across time and electrodes
"""
from scipy.stats import wilcoxon
from Ttests_perm_arthur import ttest_perm
from os.path import join
import numpy as np
from brainpipe.system import study

st = study('Olfacto')
data_rep = join(st.path,'feature/PLV_all_freqs_Encoding_v=1_elecs=all/2conds/')
filenames = join(data_rep, '{}_plv_conds={}_elecs={}_{}.npz')

to_take = [1536,2560] #[0;+2s]
freqs = ['theta']#,'alpha','beta','gamma_low','gamma_high']
conds = ['no_odor','low','high']
roi1, roi2 = ['aHC','OFC_olf']

subjects_sel = np.unique([x.split('_')[0] for x in st.search(roi1+'_'+roi2,
                                                             folder=data_rep)])

for f,freq in enumerate(freqs):
    plv_mean0, plv_mean1, plv_mean2 = [], [], []
    for su in ['LEFC','SEMC']:
        print(roi1, roi2, su)
        plv0 = np.load(filenames.format(su,conds[0],
                                roi1,roi2))['plv'][f,:,:,to_take[0]:to_take[1]]
        plv1 = np.load(filenames.format(su,conds[1],
                                roi1,roi2))['plv'][f,:,:,to_take[0]:to_take[1]]
        plv2 = np.load(filenames.format(su,conds[2],
                                roi1,roi2))['plv'][f,:,:,to_take[0]:to_take[1]]
        
        plv0 = np.reshape(np.mean(plv0,axis=-1),(plv0.shape[0]*plv0.shape[1]))
        plv1 = np.reshape(np.mean(plv1,axis=-1),(plv1.shape[0]*plv1.shape[1]))
        plv2 = np.reshape(np.mean(plv2,axis=-1),(plv2.shape[0]*plv2.shape[1]))
        print(plv1.shape)
        T0, p0 = wilcoxon(plv0,plv1, alternative='less')
        T1, p1 = wilcoxon(plv0,plv2, alternative='less')
        T2, p2 = wilcoxon(plv1,plv2, alternative='less')
        print('LOW MID', T0, np.round(p0,4))
        print('LOW HIGH', T1, np.round(p1,4))
        print('MID HIGH', T2, np.round(p2,4))
        print('mean values', np.mean(plv0), np.mean(plv1),np.mean(plv2))
        print('SD values', np.std(plv0), np.std(plv1),np.std(plv2))
        print('')
        
        plv_mean0.append(np.mean(plv0)), plv_mean1.append(np.mean(plv1))
        plv_mean2.append(np.mean(plv2))
        
    print('ALL mean', np.mean(np.array(plv_mean0)),
                  np.mean(np.array(plv_mean1)),np.mean(np.array(plv_mean2)))
    print('ALL SD', np.std(plv_mean0),np.std(plv_mean1),np.std(plv_mean2))
        

-> Olfacto loaded
aHC OFC_olf LEFC
(40,)
LOW MID 1.0 0.0
LOW HIGH 3.0 0.0
MID HIGH 107.0 0.0
mean values 0.0805787230272288 0.16477548994927205 0.19477851948907904
SD values 0.03565694618445929 0.025180792805705086 0.0289414487944629

aHC OFC_olf SEMC
(15,)
LOW MID 0.0 0.0003
LOW HIGH 0.0 0.0003
MID HIGH 0.0 0.0003
mean values 0.09340901359516628 0.15987281228921224 0.31750478750084404
SD values 0.016507144810943522 0.017593264410187395 0.028846075972650032

ALL mean 0.08699386831119754 0.16232415111924214 0.25614165349496154
ALL SD 0.0064151452839687415 0.0024513388300299033 0.0613631340058825


In [45]:
pcorr = [0.0003*2,0.00001*2]
pcorr

[0.0006, 2e-05]

### Plot phase in 2 ROIs single-trial

In [ ]:
"""
Compute and plot phase differences btw ROIs by Condition
"""

from os.path import join
import numpy as np
from brainpipe.system import study
from brainpipe.feature import phase, sigfilt

st = study('Olfacto')
path_data = join(st.path, 'database/Encoding_No_Odor/')
filename = join(path_data, '{}_odor_no_odor_bipo_sel_physFT.npz')
save_path = join(st.path, 'feature/PLV_all_freqs_Encoding_v=1_elecs=all/')
savename = join(save_path, '{}_phase_cond={}_elecs={}_{}.npz')

# Parameters for the PLV
f = [[3,7]] 
fname = ['theta'] 
sf = 512
method = 'hilbert'
roi1, roi2 = 'aHC', 'OFC_olf'
conds = ['no_odor']

for su,cond in product(subjects,conds):
    mat = np.load(filename.format(su),allow_pickle=True)

    #select only subjects with HC and OFC electrodes
    new_labels = ['pPirT' if x == 'Amg' else x for x in mat['Mai_RL']]
    log_su = [1 if all(i in new_labels for i in [roi1, roi2]) else 0]
    if log_su == [1]:
        idx_HC = [i for i,lab in enumerate(new_labels) if lab == roi1]
        idx_OFC = [i for i,lab in enumerate(new_labels) if lab == roi2]
        print('>> processing', su, len(idx_HC),'elecs in '+roi1+' &',
                len(idx_OFC), 'elecs in '+roi2)

        #compute PLV metrics between aHC and OFC electrodes
        data_e1 = mat['x'][idx_HC,:,:]
        data_e2 = mat['x'][idx_OFC,:,:]
        def_phase = phase(sf=sf, npts=data_e1.shape[1], f=f, method=method, cycle=3)
        def_filt = sigfilt(sf=sf,npts=npts, f=f)

        dico_plv = {}
        dico_plv['x_aHC'] = data_e1
        dico_plv['x_OFC_olf'] = data_e2
        dico_plv['filt_OFC_olf'] = def_filt.get(data_e2)[0]
        dico_plv['filt_aHC'] = def_filt.get(data_e1)[0]
        dico_plv['ph_aHC'] = def_phase.get(data_e1)[0]
        dico_plv['ph_OFC_olf'] = def_phase.get(data_e2)[0]

        for c in ['xyz','channels','Mai_RL']:
            dico_plv[c] = mat[c]
        np.savez(savename.format(su,cond,roi1,roi2),**dico_plv)

### Plot Phase in both ROIs

In [41]:
"""
Plot PHASE values (nelecs, nfreqs, npts, ntrials)
"""
import matplotlib.pyplot as plt
from brainpipe.visual import addLines

st = study('Olfacto')
data_path = join(st.path, 'feature/PLV_all_freqs_Encoding_v=1_elecs=all/phases_No_Odor_Low_High/')
filename = join(data_path, '{}_phase_cond={}_elecs={}_{}.npz')
savename = join(data_path, 'Plot_su={}_{}_{}_val={}_elecs={}_{}.pdf')
conds = ['low','high']
roi1, roi2 = 'aHC', 'OFC_olf'
sel = [1382,2560]
val = 'filt'

subjects = ['LEFC']

for su in subjects:
    ph_roi1, ph_roi2 = [],[]
    for cond in conds:
        mat = np.load(filename.format(su,cond,roi1,roi2),allow_pickle=True)
        ph_roi1.append(mat[val+'_'+roi1])
        ph_roi2.append(mat[val+'_'+roi2])
        
    if val == 'ph':
        ph_roi1 = [x.swapaxes(0,1) for x in ph_roi1]
        ph_roi2 = [x.swapaxes(0,1) for x in ph_roi2]

    nfreqs, nelecs_roi1, npts, _ = ph_roi1[0].shape
    nfreqs, nelecs_roi2, npts, _ = ph_roi2[0].shape
        
    for e1, e2 in product(range(nelecs_roi1),range(nelecs_roi2)):
        print(e1,e2,nelecs_roi1,nelecs_roi2)
        e_ph_roi1 = [x[0,e1,sel[0]:sel[1],:] for x in ph_roi1]
        e_ph_roi2 = [x[0,e2,sel[0]:sel[1],:] for x in ph_roi2]
        e_roi1 = np.concatenate(e_ph_roi1,axis=-1)
        e_roi2 = np.concatenate(e_ph_roi2,axis=-1)
        conds_l = ['low']*e_ph_roi1[0].shape[-1]+['high']*e_ph_roi2[0].shape[-1]
        ntrials = int(np.sum([x.shape[-1] for x in e_ph_roi1]))

        nrows = int((ntrials)/5)
        time = np.arange(-0.3,2,1/512)
        
        fig, axs = plt.subplots(nrows,5,figsize=(10, 8), facecolor='w', edgecolor='k')
        fig.subplots_adjust(hspace = .9, wspace=.001)
        axs = axs.ravel()
        
        for t in range(ntrials):
            axs[t].plot(time,e_roi1[:,t],color='black') #'aHC' is in black
            axs[t].plot(time,e_roi2[:,t],color='red') #'OFC is in red'
            axs[t].set_title('{} ({}-{}) {}'.format(
                         str(t),str(e1), str(e2),conds_l[t]))
        plt.savefig(savename.format(su,roi1,roi2,val,e1,e2))
        plt.clf()
        plt.close()

-> Olfacto loaded
0 0 8 5
0 1 8 5
0 2 8 5
0 3 8 5
0 4 8 5
1 0 8 5
1 1 8 5
1 2 8 5
1 3 8 5
1 4 8 5
2 0 8 5
2 1 8 5
2 2 8 5
2 3 8 5
2 4 8 5
3 0 8 5
3 1 8 5
3 2 8 5
3 3 8 5
3 4 8 5
4 0 8 5
4 1 8 5
4 2 8 5
4 3 8 5
4 4 8 5
5 0 8 5
5 1 8 5
5 2 8 5
5 3 8 5
5 4 8 5
6 0 8 5
6 1 8 5
6 2 8 5
6 3 8 5
6 4 8 5
7 0 8 5
7 1 8 5
7 2 8 5
7 3 8 5
7 4 8 5


In [ ]:
"""
Plot RAW SIGNAL values (nelecs, nfreqs, npts, ntrials)
"""
import matplotlib.pyplot as plt
from brainpipe.visual import addLines

st = study('Olfacto')
data_path = join(st.path, 'database/Encoding_By_Cond_v=1_elecs=all/2conds/')
filename = join(data_path, '{}_phase_cond={}_elecs={}_{}.npz')
savename = join(data_path, 'Plot_su={}_{}_{}_conds=low_high.pdf')
conds = ['low', 'high']
roi1, roi2 = 'aHC', 'OFC_olf'
sel = [1382,2560] 

subjects = ['SEMC','LEFC']

for su in ['SEMC']:
    ph_roi1, ph_roi2 = [],[]
    for cond in conds:
        mat = np.load(filename.format(su,cond,roi1,roi2),allow_pickle=True)
        ph_roi1.append(mat['ph_'+roi1])
        ph_roi2.append(mat['ph_'+roi2])
    nelecs_roi1, nfreqs, npts, _ = ph_roi1[0].shape
    nelecs_roi2, nfreqs, npts, _ = ph_roi2[0].shape
        
    for e1, e2 in product(range(nelecs_roi1),range(nelecs_roi2)):
        e_ph_roi1 = [x[e1,0,sel[0]:sel[1],:] for x in ph_roi1]
        e_ph_roi2 = [x[e2,0,sel[0]:sel[1],:] for x in ph_roi2]
        e_roi1 = np.concatenate(e_ph_roi1,axis=-1)
        e_roi2 = np.concatenate(e_ph_roi2,axis=-1)
        conds = ['low']*e_ph_roi1[0].shape[-1]+['high']*e_ph_roi2[0].shape[-1]
        ntrials = int(np.sum([x.shape[-1] for x in e_ph_roi1]))
        
        nrows = int(ntrials/5)
        time = np.arange(-0.3,2,1/512)
        
        fig, axs = plt.subplots(nrows,5,figsize=(10, 8), facecolor='w', edgecolor='k')
        fig.subplots_adjust(hspace = .9, wspace=.001)
        axs = axs.ravel()
        
        for t in range(ntrials):
            axs[t].plot(time,e_roi1[:,t],color='black') #'aHC' is in black
            axs[t].plot(time,e_roi2[:,t],color='red') #'OFC is in red'
            axs[t].set_title('{} ({}-{}) {}'.format(
                         str(t),str(e1), str(e2),conds[t]))
        plt.savefig(savename.format(su,roi1,roi2))
        0/0

In [ ]:
"""
Compute statistical differences BY PATIENT fr EARLY and LATE
average PLV across time and electrodes
"""
from scipy.stats import wilcoxon
from Ttests_perm_arthur import ttest_perm
from os.path import join
import numpy as np
from brainpipe.system import study

st = study('Olfacto')
data_rep = join(st.path,'feature/PLV_all_freqs_Encoding_v=1_elecs=all/2conds/')
filenames = join(data_rep, '{}_plv_conds={}_elecs={}_{}.npz')

to_take = [1536,2560] #[0;+2s]
freqs = ['theta']#,'alpha','beta','gamma_low','gamma_high']
conds = ['early_late']
roi1, roi2 = ['OFC_olf','pPirT']

subjects_sel = np.unique([x.split('_')[0] for x in st.search(roi1+'_'+roi2,
                                                             folder=data_rep)])

for f,freq in enumerate(freqs):
    plv_mean0, plv_mean1 = np.array([]), np.array([])
    for su in subjects_sel:
        print(roi1, roi2, su)
        plv0 = np.load(filenames.format(su,conds[0],
                                roi1,roi2))['plv_E'][f,:,:,to_take[0]:to_take[1]]
        plv1 = np.load(filenames.format(su,conds[0],
                                roi1,roi2))['plv_L'][f,:,:,to_take[0]:to_take[1]]
        
        plv0 = np.reshape(np.mean(plv0,axis=-1),(plv0.shape[0]*plv0.shape[1]))
        plv1 = np.reshape(np.mean(plv1,axis=-1),(plv1.shape[0]*plv1.shape[1]))
        
        T0, p0 = wilcoxon(plv0,plv1, alternative='two-sided')
        print('Early Late', T0, np.round(p0,4))
        print('/n mean values', np.mean(plv0), np.mean(plv1))
        print('')
        